In [58]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%gui asyncio

import asyncio
import metrics_ui.loader as loader
import metrics_ui

SERVER="nats://dbs001.edgefarm.dev:4222"
EXPORT_SUBJECT = "agilis-mc-1.EXPORT.metrics"
STREAM_NAME = "bogie-network_export-stream-aggregate"
NATS_CREDS = "user.creds"

async def main(queue, ui):
    asyncio.create_task(loader.loader(queue, ui, SERVER, NATS_CREDS, STREAM_NAME, EXPORT_SUBJECT))

queue = asyncio.Queue()
ui = metrics_ui.ui.MetricsUi(queue)

await main(queue, ui)
display(ui)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


MetricsUi(children=(Box(children=(DatetimePicker(value=None), Dropdown(description='Time range', index=3, opti…

loader loop loading from 2022-09-06 10:47:42.817538+02:00
load_from_time2022-09-06 10:47:42.817538+02:00


In [59]:
ui.draw()


draw
draw_lte


MetricsUi(children=(Box(children=(DatetimePicker(value=datetime.datetime(2022, 9, 6, 8, 0, tzinfo=datetime.tim…

In [54]:
import matplotlib.pyplot as plt
output = ui.plot_outputs[0]
df = ui.df
with output:
    fig, ax = plt.subplots()
    line = ax.plot(df["ts"], df["cellular_strength"])
    ax.set_title("LTE")
    ax.set_ylabel("Strength [%]")
    ax.set_xlabel("Time")
    plt.show(fig)

In [38]:
ui.draw()

draw
draw_lte


In [56]:
display(ui)

MetricsUi(children=(Box(children=(DatetimePicker(value=datetime.datetime(2022, 9, 6, 8, 0, tzinfo=datetime.tim…